In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# try running simulation 
import trecs
from trecs.models import ContentFiltering, PopularityRecommender
from trecs.components import Users, Items
from trecs.metrics import HomogeneityMeasurement, JaccardSimilarity
#from trecs.utils import normalize_matrix

In [16]:
n_users = 100
k_factors = 20 #k factors = 20 in Chaney et al., 2018
n_items = 200

In [18]:
# generate user vector
generator = np.random.default_rng()
#user_params = np.random.dirichlet(np.ones(20), size=10) * 10
#item_params = np.random.dirichlet(np.ones(20) * 100, size=10) * 0.1
user_params = np.random.dirichlet(np.ones(k_factors)) * 10 #mu_rho = 10*~mu_rho from Chaney
item_params = np.random.dirichlet(np.ones(k_factors) * 100) * 0.1 #mu_alpha = 0.1*~mu_alpha from Chaney
# do conversion from paper to fiure out the utility percentage visible to users
mu_n = 0.98
sigma = 1e-5
alpha = ((1-mu_n) / (sigma**2) - (1/mu_n)) * mu_n**2
beta = alpha * (1/mu_n - 1)
# each element in users is the users vector in one simulation
#users = []
#items = []
#true_utilities = []
#known_util_perc = []

#for sim_index in range(user_params.shape[0]):
#users.append(np.random.dirichlet(user_params[:], size=n_users)) # 100 users
#items.append(np.random.dirichlet(item_params[:], size=n_items)) # 200 items
#true_utilities.append(users[-1] @ items[-1].T)
#known_util_perc.append(np.random.beta(alpha, beta, size=(n_users, n_items)))

users=np.random.dirichlet(user_params[:], size=n_users) # 100 users
items=np.random.dirichlet(item_params[:], size=n_items) # 200 items
true_utilities=users[-1] @ items[-1].T
known_util_perc=np.random.beta(alpha, beta, size=(n_users, n_items))
    
# print shape
print(users.shape) # we should see 100 users with 20 attributes
print(items.shape) # we should see 1000 items with 20 attributes

(100, 20)
(200, 20)


In [14]:
len(known_util_perc)

1

In [19]:
# basically just recommends items based on the estimates of user preferences!
# this will form the basis of our "ideal" recommender
class IdealRecommender(ContentFiltering):
    def _update_user_profiles(self, interactions):
        # do not change users_hat! 
        pass

In [23]:
ideal_recs = [] # store models
cfs = []
#pops = []
#randoms = []

#for i in range(10):
true_user_representation = users # underlying true preferences
true_scores = true_utilities
noisy_scores = known_util_perc * true_utilities

# generate random pairs for evaluating jaccard similarity
pairs = [np.random.choice(100, 2, replace=False) for _ in range(500)]

u = Users(actual_user_scores = noisy_scores, size=(n_users, n_factors), num_users=n_users) # each user interacts with items based on their (noisy) knowledge of their own scores
# recommender has the ideal item representation and ideal user representation
#ideal_rec = IdealRecommender(item_representation=items.T, user_representation=true_user_representation)
#ideal_rec.add_metrics(HomogeneityMeasurement(), JaccardSimilarity(pairs))
#ideal_rec.run(timesteps=1000, train_between_steps=False)
#ideal_recs.append(ideal_rec)

# # make sure nothing weird happened during simulation
#np.testing.assert_array_equal(ideal_rec.users_hat, true_user_representation)
#np.testing.assert_array_equal(ideal_rec.items_hat, items.T)
#np.testing.assert_array_equal(u.actual_user_scores, known_util_perc * true_utilities)
#utility_ratio = true_utilities / ideal_rec.predicted_scores
#np.testing.assert_array_almost_equal(np.ones(ideal_rec.predicted_scores.shape), utility_ratio / 
                                      #utility_ratio[:][:, np.newaxis]) #This fails. May be a consequence of only doing one run

# # normal cf model for comparison

# # model only gets item attributes, doesn't know the user preferences
# cf = ContentFiltering(item_representation=items.T, actual_user_scores=u)
# cf.add_metrics(HomogeneityMeasurement(), JaccardSimilarity(pairs))
# cf.startup_and_train(timesteps=50)

# cf.run(timesteps=950)
# cfs.append(cf)

# # popularity model
# p = PopularityRecommender(actual_item_representation=items.T, actual_user_representation=u, num_items=200, 
#                           num_users=100)
# p.add_metrics(HomogeneityMeasurement(), JaccardSimilarity(pairs))
# p.startup_and_train(timesteps=50)
# p.run(timesteps=950)
# pops.append(p)

# # random recommender
# r = RandomRecommender(item_representation=items.T, actual_user_scores=u, num_items=200, num_users=100)
# r.add_metrics(HomogeneityMeasurement(), JaccardSimilarity(pairs))
# r.startup_and_train(timesteps=50)
# r.run(timesteps=950)
# randoms.append(r)

100%|██████████| 1000/1000 [00:03<00:00, 254.93it/s]


AssertionError: 
Arrays are not almost equal to 6 decimals

(shapes (100, 200), (100, 100, 200) mismatch)
 x: array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],...
 y: PredictedScores([[[1.000000e+00, 1.000000e+00, 1.000000e+00, ...,
                   1.000000e+00, 1.000000e+00, 1.000000e+00],
                  [9.164266e+00, 1.212100e+00, 1.555044e-07, ...,...

In [ ]:
# normal cf model for comparison

# model only gets item attributes, doesn't know the user preferences
cf = ContentFiltering(item_representation=items.T, actual_user_scores=u)
cf.add_metrics(HomogeneityMeasurement(), JaccardSimilarity(pairs))
cf.startup_and_train(timesteps=50)

cf.run(timesteps=950)
cfs.append(cf)

# # popularity model
# p = PopularityRecommender(actual_item_representation=items.T, actual_user_representation=u, num_items=200, 
#                           num_users=100)
# p.add_metrics(HomogeneityMeasurement(), JaccardSimilarity(pairs))
# p.startup_and_train(timesteps=50)
# p.run(timesteps=950)
# pops.append(p)

# # random recommender
# r = RandomRecommender(item_representation=items.T, actual_user_scores=u, num_items=200, num_users=100)
# r.add_metrics(HomogeneityMeasurement(), JaccardSimilarity(pairs))
# r.startup_and_train(timesteps=50)
# r.run(timesteps=950)
# randoms.append(r)

In [24]:
utility_ratio

PredictedScores([[2.10374547e-04, 1.30183492e-04, 5.01425164e+01, ...,
                  1.60163182e-06, 1.86703677e-05, 6.43798385e-01],
                 [1.92792840e-03, 1.57795407e-04, 7.79738263e-06, ...,
                  1.29028304e-05, 7.70325235e-04, 7.79733551e-06],
                 [3.22800411e-03, 1.15508303e-03, 1.90815989e+02, ...,
                  5.65772661e-05, 4.10146749e-03, 8.86398686e-01],
                 ...,
                 [7.34230557e-04, 3.57940792e-04, 2.67022635e+01, ...,
                  1.05646211e-05, 1.81160222e-03, 6.22781584e-01],
                 [3.57068595e-03, 1.75995229e-04, 8.48052618e-07, ...,
                  2.72557444e-05, 7.81532580e-04, 8.48052098e-07],
                 [6.42146820e-04, 1.43641112e-03, 4.10974088e+01, ...,
                  1.09871508e-05, 3.89063302e-03, 5.38371061e-01]])